In [1]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
import tensorflow as tf
import numpy as np
if __name__ == "__main__":
    sys.path.append("../../")
from make_anchors_orig import make_anchors, make_anchors_dict
from unpack import unpack_net_output
from notebooks_src.configs import configs

from notebooks_src.postprocessing.utils import sort_some_lists_of_tensors

importing Jupyter notebook from make_anchors_orig.ipynb
importing Jupyter notebook from unpack.ipynb
importing Jupyter notebook from ../../notebooks_src/box_processing/tf_box_util.ipynb
importing Jupyter notebook from ../../notebooks_src/postprocessing/utils.ipynb


In [5]:
def decode(net_out):
    """takes net output or encoded ground truth label
    and returns list of boxes? and class?"""
    localizations_dict = net_out["localizations"]
    predictions_dict = net_out["predictions"]
    anchors_dict = make_anchors_dict()
    bboxes, scores = bboxes_decode(localizations_dict, anchors_dict, predictions_dict)
    return bboxes, scores
    

In [6]:
def flatten_predictions(predictions):
    pshape = predictions.get_shape()
    return tf.reshape(predictions,shape=[pshape[0].value,-1, configs["num_classes"] + 1])

def bboxes_decode(localizations_dict, anchors_dict, predictions_dict):
    """Compute the relative bounding boxes from the SSD net features and
    reference anchors bounding boxes.
    Arguments:
      feat_localizations: List of Tensors containing localization features.
      anchors: List of numpy array containing anchor boxes.

    Return:
      List of Tensors Nx4: ymin, xmin, ymax, xmax
    """
    bboxes = []
    scores = []
    for fmap_size in anchors_dict.keys():
        anchors = anchors_dict[fmap_size]
        localizations = localizations_dict[fmap_size]
        predictions = flatten_predictions(predictions_dict[fmap_size])
        scores.append(predictions)
        bboxes.append(_bboxes_decode(localizations,anchors))
    
    scores = tf.concat(scores,axis=1)
    bboxes = tf.concat(bboxes,axis=1)
    #ymin, xmin, ymax, xmax = tf.split(bboxes,axis=2,num_or_size_splits=4)
    return bboxes, scores
def _bboxes_decode(localizations,
                               anchors):
    """Compute the relative bounding boxes from the layer features and
    reference anchor bounding boxes.

    Arguments:
      feat_localizations: Tensor containing localization features.
      anchors: List of numpy array containing anchor boxes.

    Return:
      Tensor Nx4: ymin, xmin, ymax, xmax
    """
    #yref,xref is fy,fx,1 with href and wref being (nanchors,)
    yref, xref, href, wref = anchors
    #each loc is num_ex,fy,fx,num_anchors,
    x_loc, y_loc, w_loc, h_loc = [tf.squeeze(tens,axis=4) for tens in tf.split(localizations, axis=4, num_or_size_splits=4)]
    cx = x_loc * wref  + xref
    cy = y_loc * href  + yref
    w = wref * tf.exp(w_loc )
    h = href * tf.exp(h_loc)
    # Boxes coordinates.
    ymin = cy - h / 2.
    xmin = cx - w / 2.
    ymax = cy + h / 2.
    xmax = cx + w / 2.

    bboxes = tf.stack([ymin, xmin, ymax, xmax], axis=-1)
    bshape = bboxes.get_shape()
    #flatten the boxes here?
    bboxes = tf.reshape(bboxes,shape=(bshape[0].value,-1,4))
    return bboxes

In [7]:
if __name__ == "__main__":
    shapes = [(2, 6, 9, 54),
                     (2, 3, 5,36 ),
                     (2, 96, 144, 36),
                     (2, 24, 36, 54),
                     (2, 12, 18, 54),
                     (2, 48, 72, 54),
                     (2, 1, 1, 36)]

    y_preds = [tf.random_normal(mean=0.0,stddev=3.,shape=shape) for shape in shapes]
    net_out_dict = unpack_net_output(y_preds)
    
    #print localizations
    boxes=decode(net_out_dict)

    print boxes